## Collect data

In [ ]:
!kaggle competitions download -c aptos2019-blindness-detection

In [ ]:
!mv *.csv *.zip data/

In [ ]:
!unzip data/train_images.zip -d data/train
!unzip data/test_images.zip -d data/test

In [ ]:
!rm data/*.zip